In [1]:
import glob
import os
import nibabel as nib
import numpy as np
from scipy.ndimage.interpolation import zoom
from scipy import ndimage
import pickle
import nibabel as nib
import random
from scipy.ndimage.interpolation import zoom
from tqdm import tqdm
import matplotlib.pyplot as plt

from torchvision import transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchcontrib.optim import SWA

from mutant_cla_network import *
from utility_func import *

/share/apps/python3/3.6.3/intel/lib/python3.6/site-packages/h5py-2.7.1-py3.6-linux-x86_64.egg/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mutant_names = read_mutant_txt('mutant_imgs.txt')
data_base_path = '/scratch/zq415/grammar_cor/mutant_detect/mutant_cla/data'
data_nii_path = '/scratch/zq415/grammar_cor/mutant_detect/mutant_cla/bv_body_nii'
save_name = 'All_data_256_196_160_down.pickle'
# 'All_data_256_196_160.pickle'
# save_name = 'All_bv_data_64_64_40_down.pickle'
# save_name = 'All_body_data_128_98_80_down.pickle'

In [3]:
if os.path.exists(os.path.join(data_base_path,save_name)):
    with open(os.path.join(data_base_path, save_name), "rb") as input_file:
        all_train_data = pickle.load(input_file)
else:
    files = glob.glob(data_nii_path + '/*.nii')
    all_train_data = {}
    for i in tqdm(range(len(files))):
        base_name = os.path.basename(files[i])[:-17]
        label = nib.load(files[i])
        ori_label = np.uint8(label.get_data()[::2,::2,::2])
        print(ori_label.shape)
        all_train_data[base_name] = ori_label
    if not os.path.exists(os.path.join(data_base_path,save_name)):
        save_file = open(os.path.join(data_base_path, save_name),'wb')
        pickle.dump(all_train_data,save_file)
        save_file.close() 

In [4]:
mutant_group = [(9,10), (12,13,14), (16,17,18,19), (36,37), (38,39), (42,43), (44,45,46,47), (48,49,50,51), (52,53), (54,55),
(56,57,58), (59,60,61), (63,64), (66,67,68), (69,70,71), (73,74), (75,76), (77,78,79), (80,81,82,83,84,85,86,87),
(89,90), (91,92), (93,94), (95,96,97), (98,99), (100,101,102)]
group_list = []
for one_group in mutant_group:
    for ii in range(len(one_group)):
        group_list.append(one_group[ii])
single_mutant = [i for i in range(len(mutant_names)) if i not in group_list]

test_mut_names = []
for i in range(2,len(mutant_group),6):
    for ii in range(len(mutant_group[i])):
        test_mut_names.append(mutant_names[mutant_group[i][ii]])

for i in range(2,len(single_mutant),6):
    test_mut_names.append(mutant_names[single_mutant[i]])

print(len(test_mut_names),test_mut_names)

17 ['20170207_En1_E13_E12a_Mut', '20170207_En1_E13_E12b_Mut', '20170207_En1_E13_E12c_Mut', '20170207_En1_E13_E12d_Mut', '20170727_En1_E13_E1a_Mut', '20170727_En1_E13_E1b_Mut', '20161222_En1_E13_E1a_Mut', '20161222_En1_E13_E1b_Mut', '20161222_En1_E13_E1c_Mut', '20180202_En1_E14_E4a_mut', '20180202_En1_E14_E4b_Mut', '20171211_En1_E10_E4b', '20170705_En1_E12_E1a_filt', '20171004_En1_E13_E5b_Mut', '20171009_En1_E12_E11a', '20161219_En1_E10_E5Ma', '20161223_En1_E14_E1a_Mut']


In [5]:
# #####for bv or body
# train_data = []
# test_data = []
# seed_num = 0
# for name in all_train_data:
#     if name in mutant_names:
#         if name in test_mut_names:
#             test_data.append((all_train_data[name]-0.5, 0))
#             print('mutant: ',name)
#         else:
#             train_data.append((all_train_data[name]-0.5,0))
#     else:
#         random.seed(seed_num*8)
# #         random.seed(seed_num*9)
#         seed_num += 1
#         if random.uniform(0,1) < 0.16:
#             test_data.append((all_train_data[name]-0.5,1))
#             print('normal: ',name)
#         else:
#             train_data.append((all_train_data[name]-0.5,1))

# print(len(test_data))
# print(len(train_data))

In [6]:
mutant_num = 0
normal_num = 0

for name in all_train_data:
    if name in mutant_names:
        mutant_num += 1
    else:
        normal_num += 1
        
print(mutant_num, normal_num)

102 440


In [7]:
#####for bv and body
train_data = []
test_data = []
seed_num = 0

for name in all_train_data:
    if name in mutant_names:
        if name in test_mut_names:
            test_data.append((all_train_data[name]-1.0, 0))
            print('mutant: ',name)
        else:
            train_data.append((all_train_data[name]-1.0,0))
    else:
        random.seed(seed_num*8)
#         random.seed(seed_num*9)
        seed_num += 1
        if random.uniform(0,1) < 0.16:
            test_data.append((all_train_data[name]-1.0,1))
            print('normal: ',name)
        else:
            train_data.append((all_train_data[name]-1.0,1))

print(len(test_data))
print(len(train_data))

normal:  20180228_En1_E13p5_Ext_E1a
normal:  20180302_En1_E14.5_Ext_E11a_reg
normal:  20171212_En1_E11_E8a
normal:  20180202_En1_E14_E12a
normal:  20171003_En1_E13_E8a_reg
normal:  20170129_En1_E11_E2a_reg
normal:  20170206_En1_E12_E3a
normal:  20170320_En1_E11_E6a
normal:  20180205_En1_E11_E1c_reg
mutant:  20180202_En1_E14_E4a_mut
normal:  20180302_En1_E14.5_Ext_E8a_reg
normal:  20180904_En1_E11p5_4a_007
mutant:  20170727_En1_E13_E1a_Mut
normal:  20170313_EN1_E10_E3b_reg_filt
normal:  20161207_En1_E14_E4a
normal:  20170619_En1_E12_E2a
normal:  20180202_En1_E14_E1a
normal:  20180226_En1_E11p5_Em2a
normal:  20161116_En1_E12_E6a_reg
normal:  20161222_En1_E13_E2a
normal:  20171002_En1_E11_E1a
normal:  20170315_En1_E12_E2b
mutant:  20161222_En1_E13_E1c_Mut
normal:  20170331_En1_E14p_E2a
normal:  20170705_En1_E12_E5a_filt
normal:  20161116_En1_E12_E2a_reg
normal:  20170619_En1_E12_Ex_E12a_reg
normal:  20180226_En1_E11p5_Em1a
mutant:  20170207_En1_E13_E12a_Mut
normal:  20170314_En1_E11_E4b
m

In [9]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# net = VGG_net()
# #net.apply(weight_init)
# if torch.cuda.device_count() > 1:
#     print("Let's use", torch.cuda.device_count(), "GPUs!")
#     net = nn.DataParallel(net)
# net.to(device)
# print("There are {} parameters in the model".format(count_parameters(net)))

# num_epochs = 120
# criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor([3.5,1.0]).to(device))

# # optimizer_base = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.00001)
# # optimizer = SWA(optimizer_base, swa_start=60, swa_freq=5)

# optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.00001)
# print('choose SGD as optimizer')
# #optimizer = optim.Adam(net.parameters(), lr=args.lr*10, weight_decay=0.00001)
# #print('choose Adam as optimizer')

# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=70, gamma=0.5)

# for epoch in range(num_epochs):
#     scheduler.step()
    
#     Mouse_dataset = Mouse_sub_volumes(train_data)#transform=transforms.Compose([Flip()])
#     dataloader = DataLoader(Mouse_dataset, batch_size=8, shuffle=True, num_workers=4, drop_last = True)
#     train(net, device, dataloader, optimizer, criterion, epoch)
#     if (epoch + 1) % 10 == 0:
#         print('epoch {} train accuracy: '.format(epoch+1))
#         train_Mouse_dataset = Mouse_sub_volumes(train_data)
#         train_dataloader = DataLoader(train_Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)
#         train_dic = test(net, device, train_dataloader)
#         get_confusion_matrix(train_dic)
        
#         print("-------------------")
#         print('epoch {} test accuracy: '.format(epoch+1))
#         test_Mouse_dataset = Mouse_sub_volumes(test_data)
#         test_dataloader = DataLoader(test_Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)
#         test_dic = test(net, device, test_dataloader)
#         get_confusion_matrix(test_dic)
        
#         torch.save(net.state_dict(), './model/mut_clas_2020_04_01_e{}_body.pth'.format(epoch+1))
        
# train_test_data = [train_data, test_data]
# save_file = open('./model/mut_clas_2020_04_01_e{}_body_data.pickle'.format(epoch+1),'wb')
# pickle.dump(train_test_data, save_file)
# save_file.close()

Let's use 2 GPUs!
There are 355214 parameters in the model
choose SGD as optimizer
epoch 1, batch 10, current loss 0.703300940990448
epoch 1, batch 20, current loss 0.6585101664066315
epoch 1, batch 30, current loss 0.6865073204040527
epoch 1, batch 40, current loss 0.7826775312423706
epoch 1, batch 50, current loss 0.719043105840683
epoch 2, batch 10, current loss 0.7033322274684906
epoch 2, batch 20, current loss 0.6509332716464996
epoch 2, batch 30, current loss 0.649322235584259
epoch 2, batch 40, current loss 0.7562605082988739
epoch 2, batch 50, current loss 0.6646083712577819
epoch 3, batch 10, current loss 0.6654145181179046
epoch 3, batch 20, current loss 0.6608971148729325
epoch 3, batch 30, current loss 0.6801113188266754
epoch 3, batch 40, current loss 0.68841752409935
epoch 3, batch 50, current loss 0.6764464497566223
epoch 4, batch 10, current loss 0.6558421969413757
epoch 4, batch 20, current loss 0.6336130678653717
epoch 4, batch 30, current loss 0.6792974591255188
epoc

KeyboardInterrupt: 

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = VGG_net()
#net.apply(weight_init)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)
net.to(device)
print("There are {} parameters in the model".format(count_parameters(net)))

In [ ]:
def test_with_probability(model, device, test_loader):
    model.eval()
    correct_num = 0
    total_num = 0
    positive_correct=0
    positive_num=0
    negative_correct=0
    negative_num=0
    
    true_predicted_labels = []
    
    with torch.no_grad():
        for i_batch, sample_batched in enumerate(test_loader):
            inputs, labels = sample_batched['image'], sample_batched['label']  
            inputs = inputs.to(device)
            # forward + backward + optimize
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            outputs = F.softmax(outputs)
            true_predicted_labels.append((labels.numpy(), predicted.cpu().numpy(), outputs.cpu().numpy()[0,0], outputs.cpu().numpy()[0,1]))
            correct_num+=np.sum(predicted.cpu().numpy()==labels.numpy())
            total_num+=len(labels)
            positive_correct+=np.sum(predicted.cpu().numpy()*labels.numpy())
            positive_num+=np.sum(labels.numpy())
            negative_correct+=np.sum((1-predicted.cpu().numpy())*(1-labels.numpy()))
            negative_num+=np.sum(1-labels.numpy())
            
    print('total_num:{}, test accuracy:{}, positive_acc:{}, negative_acc:{}'.format(total_num,
                                                                                   correct_num/total_num,
                                                                                    positive_correct/positive_num,
                                                                                    negative_correct/negative_num
                                                                                    ))
    return true_predicted_labels

# load a trained model and test to get the confusion matrix

In [ ]:
net.load_state_dict(torch.load('./model/mut_clas_2020_03_11_e100_bv_body.pth'))

with open('./model/mut_clas_2020_03_11_e100_bv_body_data.pickle', 'rb') as input_file:
    train_data, test_data = pickle.load(input_file)

In [ ]:
print('train accuracy: ')
Mouse_dataset = Mouse_sub_volumes(train_data)
train_dataloader = DataLoader(Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)
train_dic = test_with_probability(net, device, train_dataloader)

In [ ]:
print('test accuracy: ')
Mouse_dataset = Mouse_sub_volumes(test_data)
test_dataloader = DataLoader(Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)
test_dic = test_with_probability(net, device, test_dataloader)

In [ ]:
cross_table = np.zeros([2,2])
mut_to_nor = []
nor_to_mul = []
for i in range(len(test_dic)):
    if test_dic[i][0] ==0 and test_dic[i][1] ==0:
        cross_table[0,0] += 1
    elif  test_dic[i][0] ==0 and test_dic[i][1] ==1:
        cross_table[0,1] += 1
        mut_to_nor.append(i)
    elif test_dic[i][0] ==1 and test_dic[i][1] ==0:
        cross_table[1,0] += 1
        nor_to_mul.append(i)
    elif test_dic[i][0] ==1 and test_dic[i][1] ==1:
        cross_table[1,1] += 1
print(cross_table)
print(mut_to_nor)
print(nor_to_mul)

# Create the saliency map using gradeint backpropogation

In [ ]:
def compute_saliency_maps(X, y, model):
    """
    Compute a class saliency map using the model for images X and labels y.

    Input:
    - X: Input images; Tensor of shape (N, 3, H, W)
    - y: Labels for X; LongTensor of shape (N,)
    - model: A pretrained CNN that will be used to compute the saliency map.

    Returns:
    - saliency: A Tensor of shape (N, H, W) giving the saliency maps for the input
    images.
    """
    # Make sure the model is in "test" mode
    model.eval()
    
    # Make input tensor require gradient
    X.requires_grad_()
    ##############################################################################
    # Perform a forward and backward pass through the model to compute the gradient 
    # of the correct class score with respect to each input image. You first want 
    # to compute the loss over the correct scores (we'll combine losses across a batch
    # by summing), and then compute the gradients with a backward pass.
    ##############################################################################
    scores = model(X)
    
    # Get the correct class computed scores.
    scores = scores.gather(1, y.view(-1, 1)).squeeze()  
    
    # Backward pass, need to supply initial gradients of same tensor shape as scores.
    scores.backward(torch.tensor(10.0).cuda(device))
    
    # Get gradient for image.
    saliency = X.grad.data
    
    # Convert from 3d to 1d.
    saliency = saliency.abs()
    saliency = saliency.squeeze()
    ##############################################################################
    return saliency

In [ ]:
test_Mouse_dataset = Mouse_sub_volumes(test_data)
test_dataloader = DataLoader(test_Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)

for i_batch, sample_batched in enumerate(test_dataloader):
    inputs, labels = sample_batched['image'], sample_batched['label']  
    inputs = inputs.to(device)
    labels = labels.to(device)
    saliency = compute_saliency_maps(inputs, labels, net)
    
    max_value = torch.max(saliency)
    print(torch.max(saliency), torch.min(saliency))
    saliency[saliency >= (max_value*0.2)] = 100
    saliency[saliency < (max_value*0.2)] = 0
    saliency *= 0.01
    
    img_nft = nib.Nifti1Image(np.squeeze(inputs.cpu().detach().numpy()+1.0),np.eye(4))
    img_save_data_path = './saliency_map/img_label{}_{}.nii'.format(labels.cpu().numpy()[0], i_batch)
    nib.save(img_nft,img_save_data_path)
    
    print(i_batch, ' saliency num: ', np.sum(saliency.cpu().numpy()))
    saliency_nft = nib.Nifti1Image(np.squeeze(saliency.cpu().numpy()),np.eye(4))
    saliency_save_data_path = './saliency_map/salency_label{}_{}.nii'.format(labels.cpu().numpy()[0], i_batch)
    nib.save(saliency_nft,saliency_save_data_path)
    
# 5,6,8,9,10,11,12,33,27